In [6]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

from nemosis import static_table, dynamic_data_compiler
import plotly.express as px

raw_data_cache = '/Volumes/T7/NEMO_data'

pd.set_option('display.max_columns', None)

# Definition
Residual demand: RD_i(p) = Total_Demand - Sum(Supply_j(p)) for all j != i

Where:

Total_Demand is the perfectly inelastic balancing demand in that auction
Supply_j(p) are the supply bids submitted by all other firms j ≠ i

# Steps
Get the total balancing demand for the auction from the data
Collect all supply bids from rival firms in the auction
Subtract rival supply from total demand to get residual demand
This creates a step function since bids are step functions
Finally, smooth this residual demand

We're again going to first start with finding residual demand for a single Origin energy coal plant on 2021-04-07 18:05:00	

In [7]:
dispatch_load = dynamic_data_compiler(
    start_time='2021/03/01 00:00:00',
    end_time='2021/04/10 00:00:00',
    table_name='DISPATCHLOAD',
    raw_data_location=raw_data_cache
)

INFO: Compiling data for table DISPATCHLOAD
INFO: Returning DISPATCHLOAD.


In [8]:
dispatch_load

,SETTLEMENTDATE,DUID,INTERVENTION,DISPATCHMODE,AGCSTATUS,INITIALMW,TOTALCLEARED,RAMPDOWNRATE,RAMPUPRATE,LOWER5MIN,LOWER60SEC,LOWER6SEC,RAISE5MIN,RAISE60SEC,RAISE6SEC,LOWERREG,RAISEREG,AVAILABILITY,RAISEREGENABLEMENTMAX,RAISEREGENABLEMENTMIN,LOWERREGENABLEMENTMAX,LOWERREGENABLEMENTMIN,SEMIDISPATCHCAP
0,2021-03-01 00:05:00,AGLHAL,0,0,0,0.00000,0.00000,720.00,720.00,0.0,0.0,0.00000,0.0000,0.0000,0.0000,0.0,0.0,175.00000,0.00,0.0,0.00,0.0,0
1,2021-03-01 00:05:00,AGLSOM,0,0,0,0.00000,0.00000,480.00,480.00,0.0,0.0,0.00000,0.0000,0.0000,0.0000,0.0,0.0,160.00000,0.00,0.0,0.00,0.0,0
2,2021-03-01 00:05:00,ANGAST1,0,0,0,0.00000,0.00000,840.00,840.00,0.0,0.0,0.00000,0.0000,0.0000,0.0000,0.0,0.0,40.00000,0.00,0.0,0.00,0.0,0
3,2021-03-01 00:05:00,APD01,0,0,0,0.00000,0.00000,0.00,0.00,0.0,0.0,0.00000,0.0001,0.0001,0.0001,0.0,0.0,0.00000,0.00,0.0,0.00,0.0,0
4,2021-03-01 00:05:00,ARWF1,0,0,0,162.50000,161.88836,600.00,1200.00,0.0,0.0,0.00000,0.0000,0.0000,0.0000,0.0,0.0,161.88836,0.00,0.0,0.00,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914971,2021-04-10 00:00:00,YENDWF1,0,0,0,11.84000,11.49800,1680.00,1680.00,0.0,0.0,0.00000,0.0000,0.0000,0.0000,0.0,0.0,11.49800,0.00,0.0,0.00,0.0,0
914972,2021-04-10 00:00:00,YWPS1,0,0,0,0.00000,0.00000,180.00,180.00,0.0,0.0,0.00000,0.0000,0.0000,0.0000,0.0,0.0,0.00000,340.25,250.0,340.25,250.0,0
914973,2021-04-10 00:00:00,YWPS2,0,0,0,0.00000,0.00000,180.00,180.00,0.0,0.0,0.00000,0.0000,0.0000,0.0000,0.0,0.0,0.00000,355.25,250.0,355.25,250.0,0
914974,2021-04-10 00:00:00,YWPS3,0,0,1,389.87469,390.00000,180.00,180.00,0.0,0.0,0.00000,0.0000,0.0000,0.0000,0.0,0.0,390.00000,385.00,250.0,385.00,250.0,0
